In [2]:
from webdriver_manager.chrome import ChromeDriverManager
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.common.exceptions import NoSuchElementException
import time

from openpyxl import Workbook

In [4]:
original_list = ["caresense2_meter_fullset"]                          # 수집하려는 품목들을 리스트로 만든다.
product_code = {"caresense2_meter_fullset" : 9355226}

In [5]:
# 위 정보를 이용해서 각 상품별 url_list를 만들어준다.
url_list = [] 
for lis in original_list:
    key = product_code[lis] 
    raw_url = f"https://www.coupang.com/np/categories/{key}?listSize=120&brand=&offerCondition=&filterType=&isPriceRange=false&minPrice=&maxPrice=&page="
    url_list.append(raw_url)